In [2]:
from __future__ import annotations
import typing
from dataclasses import dataclass
from contextlib import contextmanager
import time

In [3]:
import numpy as np

In [4]:
@contextmanager
def localize_globals(*exceptions: str, restore_values: bool = True):
    exceptions: typing.Set[str] = set(exceptions)

    old_globals: typing.Dict[str, typing.Any] = dict(globals())
    allowed: typing.Set[str] = set(old_globals.keys())
    allowed.update(exceptions)

    yield None

    new_globals: typing.Dict[str, typing.Any] = globals()

    for name in tuple(new_globals.keys()):
        if name not in allowed:
            del new_globals[name]
    
    if not restore_values:
        return
    
    new_globals.update(
        {k: v for k, v in old_globals.items() if k not in exceptions}
    )

In [5]:
class measure_time:
    _start: float
    _end: float
    
    def __init__(self):
        self._start = 0
        self._end = 0
    
    def __enter__(self) -> measure_time:
        self._start = time.perf_counter()
        return self
    
    def __exit__(self, exc_type, exc_value, traceback) -> None:
        self._end = time.perf_counter()
    
    @property
    def elapsed(self) -> float:
        return self._end - self._start
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.elapsed:.2e})"

### Power method

In [221]:
class PowerMethodSVD:
    _half_mat: np.ndarray
    _mat: np.ndarray
    _iterations: int
    _sum_eigenvals: float
    
    def __init__(self, mat: np.ndarray, iterations: int = 1000):
        assert mat.ndim == 2
        
        self._half_mat = mat
        self._mat = mat @ mat.T
        self._iterations = iterations
    
    def apply(self) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
        """
        Returns: U, S (1-dimensional), V.T
        """
        
        U: np.ndarray = np.empty((self._mat.shape[0], 0))
        S: np.ndarray = np.empty((0,))
        
        for _ in range(self._mat.shape[0]):
            vec: np.ndarray = self._compute_eigenvec()
            vec_norm: float = np.linalg.norm(vec)
            
            if vec_norm < 1e-10:
                break
            
            U = np.append(U, vec, axis=1)
            
            val: float = np.linalg.norm(self._mat @ vec) / vec_norm
            # val: float = vec.T @ self._mat.T @ vec / vec_norm ** 2
            val = np.sqrt(val)
            
            S = np.append(S, val)
            
            self._mat -= val**2 * vec @ vec.T / vec_norm ** 2
        
        V: np.ndarray = np.linalg.inv(np.diag(S)) @ U.T @ self._half_mat
        
        return U, S, V
    
    def _compute_eigenvec(self) -> np.ndarray:
        mat: np.ndarray = self._mat
        
        approx: np.ndarray = np.random.rand(mat.shape[0], 1)
        approx /= np.linalg.norm(approx)
        
        for _ in range(self._iterations):
            approx = mat @ approx
            
            norm: float = np.linalg.norm(approx)
            if np.abs(norm) < 1e-10:
                return np.zeros_like(approx)
            
            approx /= norm
        
        return approx


In [224]:
with localize_globals():
    np.random.seed(42)
    mat: np.ndarray = np.random.rand(10, 10)
    # mat: np.ndarray = np.random.rand(3, 3)
    # mat = np.diag([1., 4., 9.])
    
    res_my = PowerMethodSVD(mat).apply()
    res_np = np.linalg.svd(mat)
    
    if mat.shape[0] <= 3:
        print(res_my)
        print(res_np)
    
    assert np.allclose(res_np[0] @ np.diag(res_np[1]) @ res_np[2], mat)
    assert np.allclose(res_my[0] @ np.diag(res_my[1]) @ res_my[2], mat)

### QR method